In [1]:
# JUPYTER CELL — Build Rhino installer links from NuGet (with padding fix) and write:
#   1) docs/rhino-versions.md       (prepend newest live entry if not present)
#   2) docs/rhino-versions-all.md   (overwrite with ALL stable entries, newest→oldest)
#
# Sources:
# - NuGet V3 Registration API ("RegistrationsBaseUrl") for package metadata.  :contentReference[oaicite:1]{index=1}
# - Rhino/Grasshopper VersionNumber: major.minor.yyddd.hhmmb (yyddd encodes build date).  :contentReference[oaicite:2]{index=2}
# - NuGet version normalization trims leading zeros (we re-pad for filenames).  :contentReference[oaicite:3]{index=3}

import datetime as dt
import os, re, sys
from typing import List, Iterable, Tuple, Union
from urllib.parse import urlparse

import requests

# ------------------ CONFIG (tweak as needed) ------------------
MAJORS: List[Union[int, str]] = [7,8,9]    # track these majors (e.g., [8, 9])
LOCALE = "en-us"                        # used in filename: rhino_<locale>_<ver>.exe
MD_LATEST = "docs/rhino-versions.md"
MD_ALL    = "docs/rhino-versions-all.md"
HEAD_CHECK_LATEST = True                # require URL 200 for newest before writing
HEAD_CHECK_ALL    = True                # require URL 200 for entries in the -all file
USER_AGENT = "nuget-rhino-jupyter/1.4"
# --------------------------------------------------------------

REG_INDEX = "https://api.nuget.org/v3/registration5-semver1/rhinocommon/index.json"
STABLE_SUFFIX_RE = re.compile(r'^[0-9]+(\.[0-9]+){3}$')  # e.g., 8.24.25281.15001
BULLET_RE = re.compile(r'^\s{4}- \[.*\]\(.*\)\s*$')

def fetch_registration_index() -> dict:
    r = requests.get(REG_INDEX, timeout=30, headers={"User-Agent": USER_AGENT})
    r.raise_for_status()
    return r.json()

def versions_from_registration(reg_json: dict) -> List[str]:
    versions: List[str] = []
    for page in reg_json.get("items", []):
        items = page.get("items")
        if items is None:
            page_url = page.get("@id")
            pr = requests.get(page_url, timeout=30, headers={"User-Agent": USER_AGENT})
            pr.raise_for_status()
            items = pr.json().get("items", [])
        for leaf in items:
            ver = (leaf.get("catalogEntry") or {}).get("version")
            if ver:
                versions.append(ver)
    return versions

def parse_version_tuple(ver: str):
    parts = ver.split(".")
    return tuple(int(p) for p in parts[:4])

def list_stable_for_majors(all_versions: List[str], majors: Iterable[Union[int, str]]) -> List[str]:
    majors_set = {str(m) for m in majors}
    cands = [v for v in all_versions if STABLE_SUFFIX_RE.match(v)]
    cands = [v for v in cands if v.split(".", 1)[0] in majors_set]
    cands.sort(key=parse_version_tuple, reverse=True)
    return cands

def decode_version_date(ver: str) -> dt.date:
    # version = "8.24.25281.15001" -> yyddd = 25281 -> 2025-10-08
    yyddd = ver.split(".")[2]
    if not (yyddd.isdigit() and len(yyddd) >= 3):
        raise ValueError(f"Unexpected yyddd segment: {yyddd}")
    yy = int(yyddd[:-3])
    ddd = int(yyddd[-3:])
    year = 2000 + yy
    return dt.date(year, 1, 1) + dt.timedelta(days=ddd - 1)

def _version_for_filename(ver: str) -> str:
    """
    NuGet normalizes versions (leading zeros trimmed), but Rhino filenames
    expect yyddd and hhmmb as 5-digit chunks → re-pad both [2] and [3].
    """
    parts = ver.split(".")
    if len(parts) < 4:
        raise ValueError(f"Unexpected version: {ver}")
    parts[2] = parts[2].zfill(5)  # yyddd
    parts[3] = parts[3].zfill(5)  # hhmmb
    return ".".join(parts[:4])

def build_dujour_url(ver: str, date_obj: dt.date, locale: str = "en-us") -> str:
    ymd = date_obj.strftime("%Y%m%d")
    ver_name = _version_for_filename(ver)
    filename = f"rhino_{locale}_{ver_name}.exe"
    return f"https://files.mcneel.com/dujour/exe/{ymd}/{filename}"

def url_exists(url: str) -> bool:
    # Prefer HEAD, but fall back to GET (some CDNs behave oddly on HEAD)
    try:
        r = requests.head(url, timeout=20, allow_redirects=True, headers={"User-Agent": USER_AGENT})
        if r.status_code == 200:
            return True
        r = requests.get(url, timeout=20, stream=True, allow_redirects=True, headers={"User-Agent": USER_AGENT})
        return r.status_code == 200
    except requests.RequestException:
        return False

def ensure_newline(s: str) -> str:
    return s if s.endswith("\n") else s + "\n"

def prepend_latest(md_path: str, filename: str, url: str) -> bool:
    bullet = f"    - [{filename}]({url})"
    os.makedirs(os.path.dirname(md_path) or ".", exist_ok=True)

    if not os.path.exists(md_path):
        with open(md_path, "w", encoding="utf-8") as f:
            f.write(bullet + "\n")
        print(f"[added:newfile] {bullet}")
        return True

    with open(md_path, "r", encoding="utf-8") as f:
        content = f.read()

    if filename in content:
        print("[ok] No update needed (already present).")
        return False

    lines = content.splitlines()
    insert_at = next((i for i, ln in enumerate(lines) if BULLET_RE.match(ln)), None)

    if insert_at is None:
        if lines and lines[-1] != "":
            lines.append("")
        lines.append(bullet)
    else:
        lines.insert(insert_at, bullet)

    new_content = ensure_newline("\n".join(lines))
    with open(md_path, "w", encoding="utf-8") as f:
        f.write(new_content)

    print(f"[added] {bullet}")
    return True

def write_all(md_path_all: str, entries: List[Tuple[str, str]]) -> int:
    os.makedirs(os.path.dirname(md_path_all) or ".", exist_ok=True)
    lines = [f"    - [{fn}]({u})" for (fn, u) in entries]
    with open(md_path_all, "w", encoding="utf-8") as f:
        f.write(ensure_newline("\n".join(lines)))
    return len(entries)

def run_update(
    majors: Iterable[Union[int, str]] = MAJORS,
    locale: str = LOCALE,
    md_latest: str = MD_LATEST,
    md_all: str = MD_ALL,
    head_check_latest: bool = HEAD_CHECK_LATEST,
    head_check_all: bool = HEAD_CHECK_ALL,
):
    latest_version = latest_date_iso = latest_filename = latest_url = None
    changed_latest = False

    try:
        reg = fetch_registration_index()
    except Exception as e:
        print(f"[error] NuGet fetch failed: {e}")
        return {"changed": False, "reason": "nuget_fetch_failed"}

    versions = versions_from_registration(reg)
    stable = list_stable_for_majors(versions, majors)

    if not stable:
        print(f"::notice::No stable Rhino versions found for majors: {', '.join(map(str, majors))}.")
        count = write_all(md_all, [])
        return {"changed": False, "all_count": count}

    # Build all entries (newest → oldest), optionally require reachable URLs
    built = []
    for v in stable:
        d = decode_version_date(v)
        u = build_dujour_url(v, d, locale=locale)
        fn = os.path.basename(urlparse(u).path)
        live = url_exists(u) if head_check_all else True
        if live:
            built.append((v, d, fn, u))
        else:
            print(f"[skip] URL not reachable: {u}")

    # Write the "all" file
    all_entries = [(fn, u) for (_v, _d, fn, u) in built]
    count = write_all(md_all, all_entries)
    print(f"Wrote {count} entries to {md_all}")

    # Prepend newest to the "latest" file
    if built:
        v, d, fn, u = built[0]
        latest_version, latest_date_iso, latest_filename, latest_url = v, d.isoformat(), fn, u
        if (not head_check_latest) or url_exists(u):
            changed_latest = prepend_latest(md_latest, fn, u)
        else:
            print(f"::notice::Latest URL not reachable; skipping update: {u}")

    # Console summary
    if latest_version:
        print(f"Latest version: {latest_version}")
        print(f"Build date:     {latest_date_iso}")
        print(f"Filename:       {latest_filename}")
        print(f"URL:            {latest_url}")
    print(f"All versions written: {count}")

    return {
        "changed": changed_latest,
        "version": latest_version,
        "date": latest_date_iso,
        "filename": latest_filename,
        "url": latest_url,
        "all_count": count,
    }

# ---- Run once when you execute the cell ----
result = run_update()
result


Wrote 71 entries to docs/rhino-versions-all.md
[added:newfile]     - [rhino_en-us_8.24.25281.15001.exe](https://files.mcneel.com/dujour/exe/20251008/rhino_en-us_8.24.25281.15001.exe)
Latest version: 8.24.25281.15001
Build date:     2025-10-08
Filename:       rhino_en-us_8.24.25281.15001.exe
URL:            https://files.mcneel.com/dujour/exe/20251008/rhino_en-us_8.24.25281.15001.exe
All versions written: 71


{'changed': True,
 'version': '8.24.25281.15001',
 'date': '2025-10-08',
 'filename': 'rhino_en-us_8.24.25281.15001.exe',
 'url': 'https://files.mcneel.com/dujour/exe/20251008/rhino_en-us_8.24.25281.15001.exe',
 'all_count': 71}